In [20]:
import torch
import torchvision
import torch.nn.functional as F
import numpy as np
import os
from pathlib import Path

# Usage

In [31]:
help(torch.onnx.export)

Help on function export in module torch.onnx:

export(model, args, f, export_params=True, verbose=False, training=<TrainingMode.EVAL: 0>, input_names=None, output_names=None, aten=False, export_raw_ir=False, operator_export_type=None, opset_version=None, _retain_param_name=True, do_constant_folding=True, example_outputs=None, strip_doc_string=True, dynamic_axes=None, keep_initializers_as_inputs=None, custom_opsets=None, enable_onnx_checker=True, use_external_data_format=False)
    Export a model into ONNX format.  This exporter runs your model
    once in order to get a trace of its execution to be exported;
    at the moment, it supports a limited set of dynamic models (e.g., RNNs.)
    
    Arguments:
        model (torch.nn.Module): the model to be exported.
        args (tuple of arguments or torch.Tensor): the inputs to
            the model, e.g., such that ``model(*args)`` is a valid
            invocation of the model.  Any non-Tensor arguments will
            be hard-coded in

# export custom model

In [26]:
custom_onnx_path = './model/custom-model/model.onnx'
os.makedirs(Path(custom_onnx_path).parent.absolute(), exist_ok=True)

In [43]:
class CustomTorchModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # assume input size: (224, 224)
        self.conv1 = torch.nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1)
        self.conv2 = torch.nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1)
        self.conv3 = torch.nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1)
        self.conv4 = torch.nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1)
        self.fc1 = torch.nn.Linear(12544, 10)
    
    def forward(self, x):
        #(3, 224, 224)
        x = self.conv1(x)
        x = F.relu(x)
        #(16, 112, 112)
        x = self.conv2(x)
        x = F.relu(x)
        #(32, 56, 56)
        x = self.conv3(x)
        x = F.relu(x)
        #(64, 28, 28)
        x = self.conv4(x)
        x = F.relu(x)
        #(64, 14, 14)
        x = torch.flatten(x)
        #(12544, )
        x = self.fc1(x)
        #(10, )
        x = F.softmax(x)
        return x
    
model = CustomTorchModel()
dummy_input = torch.Tensor(np.random.random((1, 3, 224, 224)))
torch.onnx.export(model, dummy_input, custom_onnx_path)

/home/lmy/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


# export pretrained model

In [25]:
pretrained_model_path = './model/pretrained-model/model.onnx'
os.makedirs(Path(pretrained_model_path).parent.absolute(), exist_ok=True)

In [29]:
model = torchvision.models.resnet18(pretrained=True)
dummy_input = torch.Tensor(np.random.random((1, 3, 224, 224)))
torch.onnx.export(model, dummy_input, pretrained_model_path)

# Inference all converted onnx models

In [51]:
from glob import glob
import numpy as np
import onnxruntime
import onnx

# since the model is converted from pytorch, the input image should be (B, C, H, W)
# if the model is from tensorflow, then it should be (B, H, W, C)
image = np.random.random((1, 3, 224, 224)).astype(np.float32)

In [52]:
model_paths = glob('./model/**/*.onnx', recursive=True)
for model_path in model_paths:
    onnx_model = onnx.load(model_path)
    sess = onnxruntime.InferenceSession(model_path)
    result = sess.run(output_names=[onnx_model.graph.output[0].name], input_feed={onnx_model.graph.input[0].name: image})
    result = np.argmax(result)
    print('model path = {}, result={}'.format(model_path, result))

model path = ./model/pretrained-model/model.onnx, result=111
model path = ./model/custom-model/model.onnx, result=7
